# Prototipo de prompting para comprender el uso de LLM's y su interacción.
## Trabajo Fin de Grado - Universidad de Burgos - Ingeniería Informática
### Autor: Fernando Pisot Serrano fps1001@alu.ubu.es

## Objetivo
En este notebook, vamos a construir paso a paso los prompts necesarios para obtener los puntos de interés de una ciudad utilizando un modelo de lenguaje grande (LLM). Al final, el modelo deberá devolver un JSON con los puntos de interés de la ciudad especificada, incluyendo nombre, coordenadas GPS, descripción y URL.

**LangChain** es un framework para construir aplicaciones avanzadas de LLM, permite integrar memoria, herramientas y agentes en una estructura modular.<p>
**Prompt Engineering (Ingeniería de Prompts)** : involucra la creación cuidadosa de prompts, proporcionando contexto crítico que guía a respuestas apropiadas y relevantes para el proyecto.

## Modelo de LLM
Se va a usar Llama 3  para esta versión inicial utilizando una API de GroqCloud siguiendo el ejemplo de la web de meta: https://llama.meta.com/docs/how-to-guides/prompting/ 


**Configuración e intalación de Bibliotecas**

Usaremos groq para obtener acceso al modelo: Groq proporciona acceso al motor de inferenci Groq LPU que ofrece acceso a los modelos a muy alta velocidad como se puede ver en la ejecución de los prompts.


In [4]:
import sys
!{sys.executable} -m pip install groq

https://www.youtube.com/watch?v=DELOjYAtbkg&t=977s  muestra una manera de ir generando prompts.

**Cargar un Modelo y Generar Texto**

En esta sección, cargaremos un modelo de lenguaje y generaremos texto a partir de un prompt dado.

In [2]:
import os
from typing import Dict, List
from groq import Groq

os.environ["GROQ_API_KEY"] = "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"

LLAMA3_70B_INSTRUCT = "llama3-70b-8192"
LLAMA3_8B_INSTRUCT = "llama3-8b-8192"

# Usaremos el modelo pequeño para comprobar que contiene el entrenamiento necesario (Casi 5Gb).
# Si no fuese suficiente cambiaríamos al modelo superior.
DEFAULT_MODEL = LLAMA3_8B_INSTRUCT 

client = Groq()


#Las siguientes funciones sirven como manera auxiliar para formatear los mensajes de comunicación con el LLM.
def assistant(content: str):
    return { "role": "assistant", "content": content }

def user(content: str):
    return { "role": "user", "content": content }

def chat_completion(
    messages: List[Dict],
    model = DEFAULT_MODEL,
    temperature: float = 0.6,
    top_p: float = 0.9,
) -> str:
    response = client.chat.completions.create(
        messages=messages,
        model=model,
        temperature=temperature,
        top_p=top_p,
    )
    return response.choices[0].message.content
        

def completion(
    prompt: str,
    model: str = DEFAULT_MODEL,
    temperature: float = 0.6, # Este valor garantiza cierta variabilidad en las respuestas.
    top_p: float = 0.9,
) -> str:
    return chat_completion(
        [user(prompt)],
        model=model,
        temperature=temperature,
        top_p=top_p,
    )

def complete_and_print(prompt: str, model: str = DEFAULT_MODEL):
    print(f'==============\n{prompt}\n==============')
    response = completion(prompt, model)
    print(response, end='\n\n')


In [7]:

ciudad = "Barcelona"
prompt = f"Dame una lista de puntos de interés en {ciudad} "

complete_and_print(prompt)


Dame una lista de puntos de interés en Barcelona 
Barcelona! Una ciudad llena de historia, arte, arquitectura y cultura. Aquí te dejo una lista de puntos de interés en Barcelona que no te puedes perder:

**Monumentos y Edificios**

1. La Sagrada Familia (Basilica de la Sagrada Familia): La famosa iglesia diseñada por Antoni Gaudí.
2. La Catedral de Barcelona: Una de las catedrales góticas más importantes de España.
3. El Palacio de la Generalidad de Cataluña: El palacio donde se reúne el Parlamento de Cataluña.
4. La Casa Batlló: Un edificio modernista diseñado por Gaudí.
5. La Casa Milà (La Pedrera): Otro edificio modernista de Gaudí.
6. El Barceloneta: Un paseo marítimo con vistas al puerto.
7. La Torre de Collserola: Una torre de comunicaciones con vistas panorámicas de la ciudad.

**Museos**

1. El Museo Picasso: Un museo que alberga más de 4.000 obras del artista español.
2. El Museo Nacional de Arte de Cataluña (MNAC): Un museo que exhibe arte catalán desde el siglo XIII hasta el

### Explicit Instructions
Se pide al modelo instrucciones concisas para obtener una salida con buenos resultados. Según la guía de Llama 3 se buscan 3 factores: estilización, formato y restricciones: 
- Stylization
    - `Explain this to me like a topic on a children's educational network show teaching elementary students.`
    - `I'm a software engineer using large language models for summarization. Summarize the following text in under 250 words:`
    - `Give your answer like an old timey private investigator hunting down a case step by step.`
- Formatting
    - `Use bullet points.`
    - `Return as a JSON object.`
    - `Use less technical terms and help me apply it in my work in communications.`
- Restrictions
    - `Only use academic papers.`
    - `Never give sources older than 2020.`
    - `If you don't know the answer, say that you don't know.`

In [ ]:

ciudad = "Barcelona"
prompt = f"Dame una lista de puntos de interés en {ciudad} "

complete_and_print(prompt)

### Few-Shot Prompting
Esta técnica consiste en añadir ejemplos específicos de la salida que se desea generalmente genera resultados más consistentes.

In [ ]:
# Un ejemplo de respuesta que queremos obtener sería la siguiente:
plantilla_salida = """[
        {"nombre": "Sagrada Familia", "gps": "41.4036, 2.1744", "descripción": "Basílica católica diseñada por Gaudí.", "url": "https://sagradafamilia.org"},
        {"nombre": "Parc Güell", "gps": "41.4145, 2.1527", "descripción": "Parque diseñado por Gaudí.", "url": "https://parkguell.barcelona"}
    ]"""

### Role Prompting
Dar un rol al modelo sirve para que gane contexto y genere respuestas que son más problables de alcanzar cierto nivel de satisfacción del usuario.

In [4]:
# En vez de decir:
complete_and_print("Dime 3 sitios para visitar en Barcelona.")
# Mejoraría los resultados si le indicamos que el modelo cumple un rol:
complete_and_print("Tu rol es un guía turístico comprometido con el medio ambiente preocupado por la gentrificación de las ciudades y el turismo masivo. Dime 3 sitios para visitar en Barcelona.")

Dime 3 sitios para visitar en Barcelona.
Barcelona es una ciudad llena de historia, arquitectura y cultura. Aquí te dejo tres sitios que debes visitar cuando estés en Barcelona:

1. **La Sagrada Familia**: La catedral más famosa de Barcelona, diseñada por Antoni Gaudí, es un must-see cuando visitas la ciudad. La Sagrada Familia es un monumento emblemático de la arquitectura modernista y es considerada una de las siete maravillas del mundo moderno.
2. **Park Güell**: Otro emblemático lugar diseñado por Gaudí, el Parque Güell es un jardín público con vistas impresionantes de la ciudad. El parque es famoso por sus esculturas, fuentes y mosaicos, y es un lugar ideal para relajarse y disfrutar del clima mediterráneo.
3. **La Rambla**: Esta famosa calle de Barcelona es el corazón de la ciudad. La Rambla es un lugar animado donde puedes encontrar tiendas, restaurantes, cafeterías y espectáculos callejeros. Es un lugar perfecto para conocer a los barcelonenses y disfrutar de la energía de la c

Se comprueba como dandole un rol, los lugares a visitar varían.

### Tool calling
ChatOllama now offers support for native tool calling. This enables a model to answer a given prompt using tool(s) it knows about, making it possible for models to perform more complex tasks or interact with the outside world. It follows the standard LangChain.dart tools API, so you can use it in the same way as you would with other providers that support tool-calling (e.g. ChatOpenAI, ChatAnthropic, etc.).

## RAG
